# EDA RFMiD

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import os
import progressbar

## Getting Data

In [3]:
RFMID_PATH = "./Data/Raw/MultiDiseaseClassification/RfmidMultidisease"

In [4]:
DB_PATH = './DB/'
if os.path.exists(DB_PATH):
    !rm -r {DB_PATH}
    os.mkdir(DB_PATH)
else:
    os.mkdir(DB_PATH)

In [5]:
def unzip_set(zipFile: str, outputPath: str = DB_PATH):
    print(zipFile)
    zipFile = os.path.sep.join([RFMID_PATH, zipFile])
    !7z x {zipFile} -o{outputPath}

In [6]:
[unzip_set(z) for z in os.listdir(RFMID_PATH) if '.zip' in z]

Evaluation_Set.zip

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz (806C1),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./Data/Raw/MultiDiseaseClassification/RfmidMultidiseas                                                                1 file, 1564446026 bytes (1492 MiB)

Extracting archive: ./Data/Raw/MultiDiseaseClassification/RfmidMultidisease/Evaluation_Set.zip
--
Path = ./Data/Raw/MultiDiseaseClassification/RfmidMultidisease/Evaluation_Set.zip
Type = zip
Physical Size = 1564446026

      1% 16 - Evaluation_Set/Validation/110.pn                                            5% 52 - Evaluation_Set/Validation/143.pn                                            9% 89 - Evaluation_Set/Validation/177.pn                                           12% 120 - Evaluation_Set/Validation/204.p                                           12% 1    

[None, None, None]

In [7]:
for f in os.listdir(DB_PATH):
    !mv {os.path.sep.join([DB_PATH, f])}/* {DB_PATH}
    !rm -r {os.path.sep.join([DB_PATH, f])}

## Restructuring the dataset

In [8]:
def restructure_set(csv_file: str, folder: str, column: str):

    df_columns = ["ID"]
    df_columns.append(column)

    # read csv file from the DB path
    df = pd.read_csv(os.path.sep.join([DB_PATH, csv_file]))
    df.dropna(how='all', axis=0, inplace=True)

    # get ID + specified disease column
    dr_df = df.loc[:, df.columns.isin(df_columns)]

    # dictionary column:[ID]
    dir_structure = {f'{i}':list(dr_df.loc[dr_df[column] == i].pop("ID")) for i in range(2)}

    # creating a folder named as {column} containing all files in [ID]
    for k in dir_structure.keys():
        class_path = os.path.sep.join([DB_PATH, folder, k])

        if not os.path.exists(class_path):
            os.mkdir(class_path)
        
        widgets = [
            f"Building Dataset {folder}/{k}: ",
            progressbar.Percentage(),
            " ",
            progressbar.Bar(),
            " ",
            progressbar.ETA()
        ]

        
        for i, v in enumerate(dir_structure[k]):


            pbar = progressbar.ProgressBar(
                maxval=len(dir_structure[k]),
                widgets=widgets
            ).start()

            destination = os.path.sep.join([class_path, f'{v}.png'])
            origin = os.path.sep.join([DB_PATH, folder, f'{v}.png'])
            !mv {origin} {destination}

            pbar.update(i)

        pbar.finish()

In [9]:
csv_files = [f for f in os.listdir(DB_PATH) if '.csv' in f]
folders = [f for f in os.listdir(DB_PATH) if f not in csv_files]

for csv in csv_files:
    for f in folders:
        if f in csv:
            restructure_set(csv, f, 'DR')

Building Dataset Training/0: 100% |#############################| Time: 0:00:00
Building Dataset Training/1: 100% |#############################| Time: 0:00:00
Building Dataset Validation/0: 100% |###########################| Time: 0:00:00
Building Dataset Validation/1: 100% |###########################| Time: 0:00:00
Building Dataset Test/0: 100% |#################################| Time: 0:00:00
Building Dataset Test/1: 100% |#################################| Time: 0:00:00
